In [2]:
PILE_DIR="nl-en/"
from transformers import GPT2Tokenizer
import csv
import json

In [3]:
# some code to count the max number of tokens in a sentence in the dataset
import nltk
# nltk.download('punkt')

def max_tokens_in_sentence(file_path):
    max_tokens = 0

    with open(file_path, 'r') as file:
        i = 0
        for line in file:
            i +=1
            if(i%1000) == 0:
                print(i)
            if(line):
            # Tokenize the sentence
                tokens = nltk.word_tokenize(line)
                num_tokens = len(tokens)
                # Update max_tokens if current sentence has more tokens
                if num_tokens > max_tokens:
                    max_tokens = num_tokens
    
    return max_tokens

In [51]:
# Function to tokenize a sentence and return its length
def tokenize_sentence(sentence, tokenizer):
    tokens = tokenizer.encode(sentence, max_length=1024, truncation=True)
    return len(tokens)

# Function to generate the csv byte offset file from the original dataset
def generate_dataset(input_file, output_file, tokenizer):
    with open(input_file, "r", encoding="utf-8") as f:
        lines = f.readlines()

    with open(output_file, "w", newline='', encoding="utf-8") as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["exid", "fid", "line_byte_offset", "start", "end", "take_before", "take_after", "internal_offset", "size", "start_byte", "end_byte", "count"])
        
        line_byte_offset = 0
        exid = 0
        fid = 0
        for line in lines:
            # Remove leading/trailing whitespaces and newline characters
            line = line.strip()
            
            # Calculate the end position (end of sentence)
            end = len(line) - 1

            # Tokenize the sentence and get its length
            size = tokenize_sentence(line, tokenizer)
            
            # Write the row to the CSV file
            csv_writer.writerow([exid, fid, line_byte_offset, 0, end, 0, 0, 0, size, -1, -1, -1])
            
            # Update line byte offset for the next sentence
            line_byte_offset += len(line) + 1  # Add 1 for the newline character
            exid += 1  # Increment example ID for the next sentence

In [52]:
path = "/Users/annavisman/stack/RUG/CS/Year3/thesis/thesis-llm-privacy/nl-en/europarl-v7.nl-en-200.en"

output_file = "datasets/train_dataset_en_200.csv"
# Initialize the tokenizer
# use other one here?
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Generate the dataset
generate_dataset(path, output_file, tokenizer)
    
print("Dataset generated successfully!")

Dataset generated successfully!


In [42]:
def truncate_sentence(sentence, max_tokens):
    # Tokenize the sentence
    tokens = tokenizer.tokenize(sentence)
    
    # Truncate to max_tokens tokens
    truncated_tokens = tokens[:max_tokens]
    
    # Convert tokens back to string
    truncated_sentence = tokenizer.convert_tokens_to_string(truncated_tokens)
    
    return truncated_sentence

def filter_and_truncate_sentences(input_file, output_file, max_tokens):
    with open(input_file, "r", encoding="utf-8") as f_input, \
         open(output_file, "w", encoding="utf-8") as f_output:
        
        for line in f_input:
            # Remove leading/trailing whitespaces and newline characters
            sentence = line.strip()
            
            # Tokenize the sentence
            tokens = tokenizer.tokenize(sentence)
            
            # Check if the number of tokens exceeds the maximum
            if len(tokens) >= max_tokens:
                # Truncate the sentence to max_tokens tokens
                truncated_sentence = truncate_sentence(sentence, max_tokens)
                
                # Write the truncated sentence to the output file
                f_output.write(truncated_sentence + "\n")


In [44]:
# stripping the original dataset to sentences of 200 tokens
input = "nl-en/europarl-v7.nl-en.en"

output = "nl-en/europarl-v7.nl-en-200.en"

# Filter and truncate lines in the input file
filter_and_truncate_sentences(input, output, 200)

print("Filtered and truncated lines written to:", output)

Filtered and truncated lines written to: nl-en/europarl-v7.nl-en-100.en


In [4]:
# i want to know how many entries have more than 200 tokens in a sentence

def count_large_entries(csv_file, tokens):
    # Open the CSV file for reading
    with open(csv_file, "r", newline='', encoding="utf-8") as csvfile:
        csv_reader = csv.DictReader(csvfile)
        
        # Initialize a counter for large entries
        large_entry_count = 0
        
        # Iterate through each row in the CSV file
        for row in csv_reader:
            # Convert the value in the "size" column to an integer
            # this is the number of tokens in the example
            size = int(row["size"])
            
            # Check if the size is greater than or equal to the amount of tokens supplied
            if size >= tokens:
                # Increment the counter if the condition is met
                large_entry_count += 1
                
    return large_entry_count

In [5]:
csv_file = "datasets/train_dataset_en.csv"
tokens = 200

# Count the number of large entries
count = count_large_entries(csv_file, tokens)
    
print("Number of rows with size >= ", tokens, ":", count)

Number of rows with size >=  200 : 282


In [21]:
# generating a jsonlines version of dataset to work with llm benchmark code

def text_to_jsonlines(input_file, output_file):
    with open(input_file, "r", encoding="utf-8") as f_input, \
         open(output_file, "w", encoding="utf-8") as f_output:
        id = 0

        for line in f_input:
            # Remove leading/trailing whitespaces and newline characters
            line = line.strip()
            
            # Skip empty lines
            if not line:
                continue
            
            # Create a JSON object with a "text" field containing the line
            json_object = {"exid": id,
                           "text": line}
            
            # Write the JSON object to the output file as a single line
            json.dump(json_object, f_output, ensure_ascii=False)
            f_output.write('\n')
            id += 1

In [22]:
input_file = "nl-en/europarl-v7.nl-en-200.en"
output_file = "nl-en/europarl-v7.nl-en-200.en.jsonl"

# Convert the plain text dataset to JSON Lines format
text_to_jsonlines(input_file, output_file)
    
print("Conversion completed successfully!")

Conversion completed successfully!


In [23]:
# running llm benchmark dataset split code to generate prefixes and suffixes

!python load_dataset.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


--- Logging error ---
Traceback (most recent call last):
  File "/opt/miniconda3/envs/torch/lib/python3.11/logging/__init__.py", line 1110, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/torch/lib/python3.11/logging/__init__.py", line 953, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/torch/lib/python3.11/logging/__init__.py", line 687, in format
    record.message = record.getMessage()
                     ^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/torch/lib/python3.11/logging/__init__.py", line 377, in getMessage
    msg = msg % self.args
          ~~~~^~~~~~~~~~~
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Users/annavisman/stack/RUG/CS/Year3/thesis/thesis-llm-privacy/load_dataset.py", line 54, in <module>
    logger.info("===== Starting dataset token split generation for language", LANGUAGE, "with token length", EXAMPLE_TOKEN_LEN, "===

In [24]:
import numpy as np

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")

# convert suffixes to jsonlines format
def generations_to_jsonl(output_file_path: str, data: np.ndarray):
    """Converts the tokenized data to a JSONL file at `path`."""

    with open(output_file_path, "w", encoding="utf-8", newline='') as file:
        id = 0
        for row in data:
            # Convert token IDs to strings
            # replace token space character with empty string
            decoded_string = tokenizer.decode(row, skip_special_tokens=True).replace('Ġ', '').replace("Ä", '')
            line = decoded_string.strip()

            # Skip empty lines
            if not line:
                continue

            # Create a JSON object with a "text" field containing the line
            json_object = {"exid": id,
                           "text": line}

            # Write the JSON object to the output file as a single line
            json.dump(json_object, file, ensure_ascii=False)
            file.write("\n")
            id += 1

    print("Decoded strings saved to: %s", str(output_file_path))

In [25]:
output_file = "datasets/en/200/train_prefix.jsonl"
data = np.load("datasets/en/200/train_prefix.npy")

generations_to_jsonl(output_file, data)

Decoded strings saved to: %s datasets/en/200/train_prefix.jsonl


In [17]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")

json_object = {"text": "The next item is the oral question to the Commission (B5-0206/2000) by Mr Lannoye, Mrs Auroi, Mr Bouwman, Mr Bowe, Mrs Cerdeira Morterero, Mrs Corbey, Mr Costa Paolo, Mr Deprez, Mr Desama, Mrs González Álvarez, Mrs Guy-Quint, Mr Izquierdo Collado, Mr Jonckheer, Mrs Korhola, Mr Kreissl-Dörfler, Mrs Lienemann, Mrs Lucas, Mrs McKenna, Mrs Maes, Mr Martínez Martínez, Mr Papayannakis, Mrs Patrie, Mr Arvidsson, Mr Puerta, Mr Ries, Mr Rod, Mr de Roo, Mrs Sandbæk, Mrs Schroedter, Mrs Sornosa Martínez, Mr Staes, Mr Sterckx, Mrs Terrón i C"}
tokens = tokenizer.encode(json_object["text"], max_length=1024, truncation=True)

print("Number of tokens:", len(tokens))
print("Tokens:", tokens)

# split tokens into prefix and suffix, both 50 tokens long
prefix_tokens = tokens[100:150]
suffix_tokens = tokens[150:200]
preprefix_tokens = tokens[:100]

sentence = tokenizer.decode(tokens, skip_special_tokens=True)
print("Decoded sentence:", sentence)

preprefix = tokenizer.decode(preprefix_tokens, skip_special_tokens=True)
print("Decoded preprefix:", preprefix)

prefix = tokenizer.decode(prefix_tokens, skip_special_tokens=True)
print("Decoded prefix:", prefix)

suffix = tokenizer.decode(suffix_tokens, skip_special_tokens=True)
print("Decoded suffix:", suffix)

Number of tokens: 200
Tokens: [464, 1306, 2378, 318, 262, 12886, 1808, 284, 262, 4513, 357, 33, 20, 12, 15, 22136, 14, 11024, 8, 416, 1770, 39004, 726, 68, 11, 9074, 317, 1434, 72, 11, 1770, 14551, 86, 805, 11, 1770, 9740, 68, 11, 9074, 17419, 2934, 8704, 10788, 567, 305, 11, 9074, 2744, 23454, 11, 1770, 18133, 11243, 14057, 11, 1770, 2129, 21107, 11, 1770, 2935, 1689, 11, 9074, 17109, 6557, 36858, 6184, 223, 6780, 19655, 11, 9074, 13145, 12, 4507, 600, 11, 1770, 28493, 421, 959, 4598, 7778, 4533, 11, 1770, 5966, 694, 258, 263, 11, 9074, 14769, 3937, 64, 11, 1770, 25732, 747, 75, 12, 35, 30570, 69, 1754, 11, 9074, 406, 2013, 368, 1236, 11, 9074, 15257, 11, 9074, 48403, 11, 9074, 6669, 274, 11, 1770, 3981, 8836, 710, 89, 3981, 8836, 710, 89, 11, 1770, 14185, 323, 1236, 27321, 11, 9074, 3208, 5034, 11, 1770, 943, 16921, 16528, 11, 1770, 11634, 861, 64, 11, 1770, 371, 444, 11, 1770, 6882, 11, 1770, 390, 371, 2238, 11, 9074, 3837, 65, 21241, 74, 11, 9074, 47349, 276, 353, 11, 9074, 311, 12